In [ ]:
%%capture
! pip install ml-collections

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab Notebooks/WhenPigsFlyContext/baselines/DeepGaze

In [ ]:
import sys
import cv2
import time
from matplotlib import pyplot as plt
from tqdm import tqdm, trange
import numpy as np
import pandas as pd
import pickle
import random
import copy

import os
import shutil
from PIL import Image, ImageDraw
from google.colab.patches import cv2_imshow

import torch
from torch.utils.data import Dataset
from torchvision.transforms.functional import to_tensor, normalize
from torchvision import transforms

import matplotlib.pyplot as plt
import numpy as np
from scipy.misc import face
from scipy.ndimage import zoom
from scipy.special import logsumexp
import torch
import deepgaze_pytorch
from deepgaze_pytorch import modules

from naturaldesign.naturaldesign import NaturalDesign
from utils import *
sys.path.append("..")

In [ ]:
test_imagedir = "../../naturaldesign/"
dataset = NaturalDesign(test_imagedir)

In [28]:
def logsearchProcess(x, y, tg_xy, attentionMap, image_size, size, coef):
    mask_size = size
    tg_x, tg_y, w, h = tg_xy
    tg_xmax, tg_ymax = tg_x + w, tg_y + h 

    attenNP = (attentionMap[0,:,:].detach() * coef[0,:,:].detach()).numpy()
    y_fix, x_fix = y, x

    x_max_s, x_min_s, y_max_s, y_min_s = min(x_fix+mask_size//2, image_size[1]-1), max(x_fix-mask_size//2, 0), min(y_fix+mask_size//2, image_size[0]-1), max(y_fix-mask_size//2, 0)

    if x_max_s < tg_x or x_min_s > tg_xmax or y_max_s < tg_y or y_min_s > tg_ymax:
        coef[0, y_min_s:y_max_s+1, x_min_s:x_max_s+1] = 1000
        attenNP = (attentionMap[0,:,:].detach() * coef[0,:,:].detach()).numpy()
        y_fix, x_fix = np.unravel_index(attenNP.argmax(), attenNP.shape)
        return False, (x_fix, y_fix), coef

    return True, [], coef

def fixation_initialize():
    k, x_range, y_range = 4, 80, 65
    x_init, y_init = 640//2, 512//2
    random_num_x, random_num_y = random.sample(list(range(x_range)), 4), random.sample(list(range(y_range)), 4)
    x, y = [], []
    for i in range(4):
        if i < 3:
            choice_x, choice_y = random.choice([0, 1]), random.choice([0, 1])
            x_cord = int(x_init+random_num_x[i]) if choice_x == 0 else int(x_init-random_num_x[i])
            y_cord = int(y_init+random_num_y[i]) if choice_y == 0 else int(y_init-random_num_y[i])
            x.append(x_cord)
            y.append(y_cord)
        else:
            x.append(320)
            y.append(256)

    return x, y

In [31]:
fixation_initialize()

([340, 376, 284, 320], [262, 287, 202, 256])

In [ ]:
%%capture
DEVICE = 'cuda'
img_size = (512, 640)
# you can use DeepGazeI or DeepGazeIIE
model = deepgaze_pytorch.DeepGazeIII(pretrained=True).to(DEVICE)
image = face()
centerbias_template = np.load('centerbias_mit1003.npy')
# rescale to match image size
centerbias = zoom(centerbias_template, (image.shape[0]/centerbias_template.shape[0], image.shape[1]/centerbias_template.shape[1]), order=0, mode='nearest')
# renormalize log density
centerbias -= logsumexp(centerbias)
centerbias_tensor = torch.tensor([centerbias]).to(DEVICE)
centerbias_tensor = transforms.Resize(img_size)(centerbias_tensor)

In [32]:
deepgaze3_res, size = [], 100
scanpath = {}

for i in trange(1, len(dataset)+1):
    img, bbox_relative = dataset[i]
    # get the target bounding box
    tg_loc = bbox_cordinates(bbox_relative, img_size[1], img_size[0])
    history_x, history_y = fixation_initialize()
    # transform img to tensor
    img_tensor = torch.tensor([img.transpose(2, 0, 1)]).to(DEVICE)
    img_tensor = transforms.Resize(img_size)(img_tensor)

    count, max_search, coef, path = 0, 999, torch.ones((1, 512, 640)), []
    while count < max_search:
        fixation_history_x = np.array(history_x)
        fixation_history_y = np.array(history_y)
        x_hist_tensor = torch.tensor([fixation_history_x[model.included_fixations]]).to(DEVICE)
        y_hist_tensor = torch.tensor([fixation_history_y[model.included_fixations]]).to(DEVICE)
        log_density_prediction = model(img_tensor, centerbias_tensor, x_hist_tensor, y_hist_tensor)

        path.append([history_x[-1], history_y[-1]])
        isTg, coordinates, coef = logsearchProcess(history_x[-1], history_y[-1], tg_loc, log_density_prediction.squeeze(0).cpu(), img_size, size, coef)
        count += 1

        if isTg: 
            break

        history_x.append(coordinates[0])
        history_y.append(coordinates[1])

    scanpath[i] = path
    print(count)
    deepgaze3_res.append(count)

  0%|          | 1/240 [00:13<52:49, 13.26s/it]

38


  1%|          | 2/240 [00:39<1:22:47, 20.87s/it]

76


  1%|▏         | 3/240 [00:48<1:01:13, 15.50s/it]

...

14


100%|█████████▉| 239/240 [23:26<00:05,  5.34s/it]

4


100%|██████████| 240/240 [23:28<00:00,  5.87s/it]

5


In [ ]:
scanpath

In [34]:
deepgaze_naturaldesign_res = {}
deepgaze_naturaldesign_res['scanpath'] = scanpath

In [35]:
with open("../results/NaturalDesign/[Fig5]NaturalDesign_deepgaze_res.pkl", "wb") as tf:
    pickle.dump(deepgaze_naturaldesign_res, tf)

In [36]:
deepgaze3_accu_ND = [0] + model_performance(deepgaze3_res, len(deepgaze3_res))
deepgaze3_accu_ND[:11]

[0,
 0.11666666666666667,
 0.17083333333333334,
 0.22916666666666666,
 0.29583333333333334,
 0.325,
 0.37083333333333335,
 0.4083333333333333,
 0.42083333333333334,
 0.4625,
 0.5083333333333333]

In [37]:
with open("../results/naturaldesign_deepgaze3_accu_performance.pkl", "wb") as tf:
    pickle.dump(deepgaze3_accu_ND, tf)